**crear un job que realizará
tuning de hiperparámetros de al menos un par de modelos:**

In [14]:
import os
import urllib
from IPython.display import Image
from zipfile import ZipFile
from azure.ai.ml.entities import AmlCompute
from azure.core.exceptions import ResourceNotFoundError

from azure.ai.ml.constants import AssetTypes, InputOutputModes
from azure.ai.ml import Input

from azure.ai.ml import MLClient
from azure.ai.ml import automl
from azure.identity import DefaultAzureCredential

from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes, InputOutputModes
from azure.ai.ml import Input

from PIL import Image
import jsonlines

import json

def get_azure_credential(subscription_id, resource_group, workspace):
    credential = DefaultAzureCredential()
    ml_client = MLClient.from_config(credential)
    ml_client =  MLClient(credential, subscription_id, resource_group, workspace)
    return ml_client

ml_client = get_azure_credential("<subcription_id>","<resource_group>","<workspace>")


Found the config file in: /config.json


In [15]:
compute_name = "gpl-cluster-nc6sv3"
dataset_parent_dir = "./data"

training_mltable_path = os.path.join(dataset_parent_dir, "training-mltable-folder")
validation_mltable_path = os.path.join(dataset_parent_dir, "validation-mltable-folder")

my_training_data_input = Input(type=AssetTypes.MLTABLE,path=training_mltable_path)
my_validation_data_input = Input(type=AssetTypes.MLTABLE,path=validation_mltable_path)


In [16]:
exp_name = "dl-car-bike-objects-hyper"

image_classfication_job = automl.image_classification(
    compute=compute_name,
    experiment_name=exp_name,
    training_data=my_training_data_input,
    validation_data=my_validation_data_input,
    target_column_name='label',
    primary_metric="accuracy",
    tags={"exp-name":"Job for image classification with car bikes object ds"}
)

image_classfication_job.set_limits(max_trials=4, max_concurrent_trials=2)
image_classfication_job.set_limits(timeout_minutes=60)




**tuning de hiperparámetros**

In [17]:
from azure.ai.ml.automl import SearchSpace
from azure.ai.ml.sweep import (Choice,Uniform,BanditPolicy,)


In [18]:

# eleccion modelos a revisar
tun_1 = SearchSpace( model_name=Choice(["vitb16r224", "yolov5"]),learning_rate=Uniform(0.001, 0.01),number_of_epochs=Choice([15, 30]), )
tun_2 = SearchSpace( model_name=Choice(["seresnext", "resnet50"]),learning_rate=Uniform(0.001, 0.01),layers_to_freeze=Choice([0, 2]),)
tunning = [tun_1, tun_2]


image_classfication_job.extend_search_space(tunning)

In [19]:
# tomade muestras random
# politicas de terminacion
limit_job = BanditPolicy(evaluation_interval=2, slack_factor=0.2, delay_evaluation=6)
image_classfication_job.set_sweep(sampling_algorithm="Random", early_termination=limit_job,)

In [20]:
returned_job = ml_client.jobs.create_or_update(image_classfication_job)

**Get training info**

In [21]:
from mlflow.tracking.client import MlflowClient

mlflow_client = MlflowClient()

job_name = returned_job.name

mlflow_parent_run =  mlflow_client.get_run(job_name)
print(mlflow_parent_run)

<Run: data=<RunData: metrics={'AUC_binary': 1.0,
 'AUC_macro': 1.0,
 'AUC_micro': 1.0,
 'AUC_weighted': 1.0,
 'accuracy': 1.0,
 'accuracy_train': 0.99062,
 'average_precision_score_binary': 1.0,
 'average_precision_score_macro': 1.0,
 'average_precision_score_micro': 1.0,
 'average_precision_score_weighted': 1.0,
 'f1_score_binary': 1.0,
 'f1_score_macro': 1.0,
 'f1_score_micro': 1.0,
 'f1_score_weighted': 1.0,
 'log_loss': 0.00252,
 'log_loss_train': 0.02232,
 'precision_score_binary': 1.0,
 'precision_score_macro': 1.0,
 'precision_score_micro': 1.0,
 'precision_score_weighted': 1.0,
 'recall_score_binary': 1.0,
 'recall_score_macro': 1.0,
 'recall_score_micro': 1.0,
 'recall_score_weighted': 1.0}, params={}, tags={'automl_best_child_run_id': 'shy_loquat_f2k12bdbvr_HD_1',
 'exp-name': 'Job for image classification with car bikes object ds',
 'fit_time_000': 'NaN',
 'is_gpu': 'True',
 'iteration_000': '0',
 'mlflow.rootRunId': 'shy_loquat_f2k12bdbvr',
 'mlflow.runName': 'shy_loquat_f2